### 数据转换：
##### 将序列数据映射为图片数据
##### 建立电极位置矩阵，将每个时间段的数据添加到矩阵的相应位置
##### 每个数据设置5个通道，分别为每个数据的不同频段

![电极位置](EEG-电极位置.png)

![电极位置](电极矩阵.png)

---
#### 导入库文件

In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from torch import nn

In [2]:
# 选取DE特征，62个电极（位置），42个时间片，5个频段（通道）-- 第一段影片（label：1）     （62*42*5）*24
feature_data = sio.loadmat('F:/SEED_IV/eeg_feature_smooth/1/1_20160518.mat')
de_LDS1 = feature_data['de_LDS1']

a_0_0 = np.array([[None,None,None,0,0,0,None,None,None],
         [None,None,0,None,None,None,0,None,None],
         [0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0],
         [0,0,0,None,0,None,0,0,0],
         [0,None,None,0,0,0,None,None,0]])
print(a_0_0.shape)
print(a_0_0)

(9, 9)
[[None None None 0 0 0 None None None]
 [None None 0 None None None 0 None None]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0]
 [0 0 0 None 0 None 0 0 0]
 [0 None None 0 0 0 None None 0]]


In [3]:
a11 = de_LDS1[:,0,0]
a12 = de_LDS1[:,0,1]
a13 = de_LDS1[:,0,2]
a14 = de_LDS1[:,0,3]
a15 = de_LDS1[:,0,4]
print(a11)
print(type(a11))

[27.79550063 27.70755329 27.70877052 25.76243677 26.43667776 26.87119573
 25.52378348 25.50432064 25.45221316 24.68049499 24.99748288 25.39045056
 26.21502666 26.92475689 25.83411012 25.52329217 25.37707268 25.4431436
 23.4182796  23.80307528 24.39529234 25.36327447 26.23362058 25.8080683
 25.02133628 24.13387879 23.12373895 23.20622596 22.66186347 23.6165822
 24.83129758 25.6194159  25.11857703 24.26400247 24.25969596 23.02365615
 22.69559715 22.50364061 23.32774884 24.44075938 25.18639365 25.12325284
 25.11109976 24.25329461 23.80442346 23.53136072 23.4148799  23.93613747
 24.6926115  25.11323317 25.24880694 25.250997   25.24537714 24.66936553
 25.03184278 25.23178332 25.38469286 25.26199833 25.25470904 25.3774774
 25.4488595  25.78729622]
<class 'numpy.ndarray'>


In [4]:
a_1_1 = a_0_0   # 第一个时间片的五个频段
a_1_2 = a_0_0
a_1_3 = a_0_0
a_1_4 = a_0_0
a_1_5 = a_0_0
k = 0
for i in range(9):
    for j in range(9):
        if a_1_1[i][j] != None:
            a_1_1[i][j] = a11[k]
            a_1_2[i][j] = a12[k]
            a_1_3[i][j] = a13[k]
            a_1_4[i][j] = a14[k]
            a_1_5[i][j] = a15[k]
            k += 1
for i in range(9):
    for j in range(9):
        if a_1_1[i][j] == None:
            a_1_1[i][j] = 0
            a_1_2[i][j] = 0
            a_1_3[i][j] = 0
            a_1_4[i][j] = 0
            a_1_5[i][j] = 0
            k += 1
            
print(a_1_1.shape)

(9, 9)


In [5]:
a_1_1

array([[0, 0, 0, np.float64(19.978601124706316),
        np.float64(19.33123134351613), np.float64(19.39072735408409), 0,
        0, 0],
       [0, 0, np.float64(18.76187136956798), 0, 0, 0,
        np.float64(18.72960465901733), 0, 0],
       [np.float64(20.43789040490055), np.float64(18.584260868272082),
        np.float64(18.586632778202723), np.float64(18.54354386501239),
        np.float64(18.260921510530846), np.float64(18.21167599428366),
        np.float64(18.30072166928622), np.float64(18.674860000679836),
        np.float64(19.147998096613183)],
       [np.float64(18.93196229534853), np.float64(18.669881868386152),
        np.float64(18.50511930849076), np.float64(18.543650859627224),
        np.float64(17.70723004140206), np.float64(18.01098182937647),
        np.float64(17.94949859802592), np.float64(18.34184729515074),
        np.float64(18.985420134927754)],
       [np.float64(18.91765202715571), np.float64(18.311309360825692),
        np.float64(18.03837970636684), np.fl

![电极位置](电极矩阵.png)


In [6]:
a_1 = np.array([a_1_1, a_1_2, a_1_3, a_1_4, a_1_5])
a_1.shape
# 64*42*5 ——>  42*(5*9*9) -----------------------------------   别人论文中输入（9*9*5）

(5, 9, 9)

In [11]:
a_1 = a_1.astype(np.float64)

In [12]:
a_1

array([[[ 0.        ,  0.        ,  0.        , 19.97860112,
         19.33123134, 19.39072735,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        , 18.76187137,  0.        ,
          0.        ,  0.        , 18.72960466,  0.        ,
          0.        ],
        [20.4378904 , 18.58426087, 18.58663278, 18.54354387,
         18.26092151, 18.21167599, 18.30072167, 18.67486   ,
         19.1479981 ],
        [18.9319623 , 18.66988187, 18.50511931, 18.54365086,
         17.70723004, 18.01098183, 17.9494986 , 18.3418473 ,
         18.98542013],
        [18.91765203, 18.31130936, 18.03837971, 17.67997513,
         18.09132819, 17.41347356, 17.76311315, 18.36409337,
         18.77965083],
        [18.43610427, 17.93022391, 17.92709198, 17.5667125 ,
         18.01634562, 17.89096302, 17.7746547 , 18.28260195,
         18.66472214],
        [18.43666239, 18.43901078, 17.92415999, 18.05219343,
         18.08936833, 18.07768723, 18.12424964, 18.73086616,
        

In [8]:
a0 = np.zeros((9,9,5))
a1 = a0.copy()
for i in range(9):
    for j in range(9):
        a1[i][j][0] = a_1_1[i][j]
        a1[i][j][1] = a_1_2[i][j]
        a1[i][j][2] = a_1_3[i][j]
        a1[i][j][3] = a_1_4[i][j]
        a1[i][j][4] = a_1_5[i][j]

In [9]:
a1

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [19.97860112, 19.97860112, 19.97860112, 19.97860112,
         19.97860112],
        [19.33123134, 19.33123134, 19.33123134, 19.33123134,
         19.33123134],
        [19.39072735, 19.39072735, 19.39072735, 19.39072735,
         19.39072735],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [18.76187137, 18.76187137, 18.76187137, 18.76187137,
         18.7

In [10]:
a1.shape

(9, 9, 5)

### 我处理的：5×9×9
### 别人论文中提到的：9×9×5
### 模型2D卷积输入：（[batch_size, channels, height, width]）